In [1]:
import pandas as  pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import numpy as np

from sklearn.model_selection import train_test_split


In [2]:
df=pd.read_csv(r"C:\Users\Admin\Downloads\Deep Learning\IMDB Dataset.csv")

In [3]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
def lowercase(str):
    sentence=str.lower()
    return sentence
df['review']=df['review'].apply(lowercase)
    

In [7]:
def removepuntuation(str):
    str=str.replace("<br />","")
    text_p = "".join([char for char in str if char not in string.punctuation])
    return text_p
df['review']=df['review'].apply(removepuntuation)

In [8]:
X=df['review']
y=df['sentiment']

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y)

In [10]:
X_train.shape,X_test.shape

((40000,), (10000,))

In [11]:
X_train

12272    finally we have before us a category iii movie...
39275    let me start off by saying that this doesnt se...
273      fulci does this man brings one of the goriest ...
47666    i was interested to see the move thinking that...
41221    an anthology is always risky business and i th...
                               ...                        
16662    professor paul steiner is doing research in ma...
630      it was 1 am in the morning and i had nothing e...
23471    simply put the movie is boring cliché upon cli...
36855    this review contains mild spoilers but not eno...
7745     bean kevin  perry uk tv creations that have ma...
Name: review, Length: 40000, dtype: object

In [12]:
tokenizer=Tokenizer(num_words=600)
tokenizer.fit_on_texts(X_train)
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)
vocab_size=len(tokenizer.word_index)+1

In [13]:
from tensorflow.keras.utils import pad_sequences
X_train=pad_sequences(X_train,padding="post",maxlen=1465)
X_test=pad_sequences(X_test,padding="post",maxlen=1465)

In [14]:
y_train=np.where(y_train=='negative',0,1)
y_test=np.where(y_test=='negative',0,1)

In [15]:
y_train

array([1, 0, 1, ..., 0, 1, 0])

In [16]:
from keras.utils import to_categorical
num_classess=2
y_train=to_categorical(y_train,num_classess)
y_test=to_categorical(y_test,num_classess)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN,Dropout,Activation,LSTM
from tensorflow.keras.optimizers import Adam


In [18]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [19]:
X_train.shape

(40000, 1465, 1)

In [20]:
X_test.shape

(10000, 1465, 1)

In [23]:
model=Sequential()
model.add(LSTM(64,input_shape=(1465,1),return_sequences=True))
model.add(LSTM(32,input_shape=(None,64)))
model.add(Dense(2,activation='softmax'))
model.summary()
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1465, 64)          16896     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 29,378
Trainable params: 29,378
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train,y_train,epochs=2,batch_size=64)

Epoch 1/2
625/625 [==============================] - 161s 249ms/step - loss: 0.6933 - accuracy: 0.5027
Epoch 2/2
625/625 [==============================] - 157s 251ms/step - loss: 0.6932 - accuracy: 0.4973


In [25]:
from sklearn.metrics import accuracy_score
y_predict=model.predict(X_test)


313/313 [==============================] - 40s 126ms/step


In [29]:
y_pred=np.argmax(y_predict,axis=1)

In [30]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [33]:
y_test=np.argmax(y_test,axis=1)

In [34]:

print(accuracy_score(y_test,y_pred))

0.5


In [35]:
y_predict

array([[0.50231814, 0.4976818 ],
       [0.50231814, 0.4976818 ],
       [0.50231814, 0.4976818 ],
       ...,
       [0.50231814, 0.4976818 ],
       [0.50231814, 0.4976818 ],
       [0.50231814, 0.4976818 ]], dtype=float32)